### **Introduction**


**A model is, abstractly:**

* A function that computes something on tensors (a forward pass)
* Some variables that can be updated in response to training

### **Setup**


In [2]:
import tensorflow as tf
from datetime import datetime
import random as ra
import tensorboard as tb

print(tb.__version__)
print(tf.__version__)

2.7.0
2.6.0


### **Defining Models and Layers in TensorFlow**
**Layers are functions with a known `mathematical structure` that can be reused and have `trainable variables`.**

In [3]:
# Creating a simple Model using tf.Module
class Module(tf.Module):
    def __init__(self, name):
        super().__init__(name= name)
        self.tr_var = tf.Variable(5.0, name= 'train_me')
        self.non_tr_var = tf.Variable(5.0, trainable=False , name= 'not_train')

    def __call__(self, x):
        return self.tr_var * x + self.non_tr_var

model = Module('SimpleModel')
model(tf.constant(3.)).numpy()

20.0

In [4]:
# Printing trainable & non_trainable variables
print('Trainable variables: ', model.trainable_variables)
print('Non-Trainable variables', model.non_trainable_variables)
print()
print('All variables: ')
for var in model.variables:
    print(var)

Trainable variables:  (<tf.Variable 'train_me:0' shape=() dtype=float32, numpy=5.0>,)
Non-Trainable variables (<tf.Variable 'not_train:0' shape=() dtype=float32, numpy=5.0>,)

All variables: 
<tf.Variable 'not_train:0' shape=() dtype=float32, numpy=5.0>
<tf.Variable 'train_me:0' shape=() dtype=float32, numpy=5.0>


> **Creating a two-layer linear layer model made out of modules.**

In [5]:
# Creating a dense layer
class Dense(tf.Module):
    def __init__(self, in_fea, out_fea,name=None):
        super().__init__(name)
        self.wei = tf.Variable(
            tf.random.normal([in_fea, out_fea], name= 'weights')
        )
        self.bias = tf.Variable(
            tf.zeros([out_fea], name='bias')
        )

    def __call__(self, x_ten):
        # Computation: Y = Matrix multiplication (x & weights) plus the model bias
        # Computation: tf.nn.relu(Y)
        y = (x_ten @ self.wei) + self.bias
        return tf.nn.relu(y)

In [6]:
class Sequential(tf.Module):
    def __init__(self, name=None):
        super().__init__(name)
        
        self.dense1 = Dense(3, 3, 'InputLayer')
        self.dense2 = Dense(3, 2, 'OutputLayer')
        
    def __call__(self, tx):
        y = self.dense1(tx)
        return self.dense2(y)

my_model = Sequential('AI_Model')
# Model acceptable shape [n x 3]
my_model(tf.random.poisson([2, 3], 5.5)) 
my_model(tf.random.normal([3, 3], mean=17, stddev=3.5)).numpy()


array([[14.620003, 99.03945 ],
       [12.422647, 99.27185 ],
       [15.690988, 63.555214]], dtype=float32)

In [7]:
# Printing Model submodules
for mod in my_model.submodules:
    print(mod)
print()

# Printing model Variables
for var in  my_model.variables: 
    print(f'var: {var.numpy()}\n')




var: [0. 0. 0.]

var: [[ 2.2870154   0.00319489  0.13522476]
 [ 0.29424453  0.25586474 -0.30452242]
 [ 0.37787178  0.84810615 -0.07773181]]

var: [0. 0.]

var: [[-0.4141421   1.5877358 ]
 [ 1.5809528   0.6924127 ]
 [-0.3139612  -0.32010743]]



* ##### **Waiting to create variables**


In [8]:
# The Input shape of the layers is dynamically inferered at the runtime by the size of the incoming tensor..
class dyn_Layer(tf.Module):
    def __init__(self, outCh, name=None):
        super().__init__(name)
        self.is_built = False 
        self.out_channels = outCh
    
    @tf.function
    def __call__(self, tx):
        if not self.is_built:
            self.is_built = True
            self.wei = tf.Variable(
                tf.random.normal([tx.shape[-1], self.out_channels]), name='Weights'
            )
            self.bias = tf.Variable(
                tf.zeros([self.out_channels], name='bias')
            )
        
        y = tx @ self.wei + self.bias
        return tf.nn.relu(y)
    
class dyn_SeqModel(tf.Module):
    def __init__(self, name=None):
        super().__init__(name)
        self.dense1 = dyn_Layer(12, 'Input_Layer')
        self.dense2 = dyn_Layer(3, 'Output_Layer')
        
    @tf.function
    def __call__(self, tx):
        tx = self.dense1(tx)
        return self.dense2(tx)

In [9]:
# Calling the dynamic model on different sets of inputs
for i in range(10):
    in_chs = ra.randint(1,15)
    print('InChanels for this Model', in_chs)
    model = dyn_SeqModel('AdaptiveModel')
    tx = tf.random.uniform([6, 6], minval=16, maxval=32)
    print(model(tx))

# Our model is working...



InChanels for this Model 5
tf.Tensor(
[[32.586563 28.136518 41.810143]
 [37.952003 33.553192 52.098286]
 [43.578575 51.636368 56.145725]
 [21.4276    0.       48.96453 ]
 [30.29971  37.51879  62.928024]
 [41.38082  11.17831  61.114265]], shape=(6, 3), dtype=float32)
InChanels for this Model 7
tf.Tensor(
[[100.25137    19.110264  144.00226  ]
 [ 85.05576    16.723856  162.82172  ]
 [ 39.593456    4.2811165  77.2881   ]
 [ 78.98489    12.434948   89.01366  ]
 [117.83304    28.318377  140.31369  ]
 [102.60702     0.        189.57448  ]], shape=(6, 3), dtype=float32)
InChanels for this Model 7
tf.Tensor(
[[  0.         0.        70.673706]
 [  0.         0.        97.84123 ]
 [  0.         0.        69.60656 ]
 [  0.         0.       135.0037  ]
 [  0.         0.       112.29971 ]
 [  0.         0.       107.42939 ]], shape=(6, 3), dtype=float32)
InChanels for this Model 6
tf.Tensor(
[[ 79.274      0.        88.99805 ]
 [ 87.23201    0.        78.11565 ]
 [ 59.740997   0.        91.46654 ]

### **Saving Weights**

In [10]:
# Create a dir to save weights
chk_path = 'Weights_checkpoint'
checkpoint = tf.train.Checkpoint(model=model)

# Using this Command, the weights are stored into the file.
checkpoint.write(chk_path)
tf.train.list_variables(chk_path)

# Creating a new Model and assigning the `SAME WEIGHTS`
new_mod = dyn_SeqModel('Dynamic_Seq2')
checkpoint2 = tf.train.Checkpoint(model=new_mod)

# Reloading the previous weights
checkpoint2.restore('Weights_checkpoint')

# Testing the new Model on previous input, should be able to produce same output
print(f'Previous Model results:\n {model(tx).numpy()}\n')
print(f'New Model result after restoring weights:\n {new_mod(tx).numpy()}')

print("It worked")


Previous Model results:
 [[0.       0.       0.      ]
 [0.       0.       0.      ]
 [0.       0.       0.      ]
 [0.       0.       0.      ]
 [5.551487 0.       0.      ]
 [0.       0.       0.      ]]

New Model result after restoring weights:
 [[0.       0.       0.      ]
 [0.       0.       0.      ]
 [0.       0.       0.      ]
 [0.       0.       0.      ]
 [5.551487 0.       0.      ]
 [0.       0.       0.      ]]
It worked


### **Saving Functions + Visualization using Tensorboard**

In [ ]:
# setting up logging
timest = datetime.now().strftime("%Y%m%d-%H%M%S")
logs_dir = 'logs/func/%s' % timest
writer = tf.summary.create_file_writer(logs_dir)

# Create new model to get fresh trace
SeqModel = dyn_SeqModel('Traced_Model')
tf.summary.trace_on(graph= True)
tf.profiler.experimental.start(logs_dir)

# Only call one tf.function while tracing
print(SeqModel(tf.random.uniform(shape=[4, 9], minval= 50, maxval= 70)))

with writer.as_default():
    tf.summary.trace_export(
        name= 'Function Trace',
        step= 0,
        profiler_outdir= logs_dir
    )

> **Starting Tensorboard to visualize tracing results**

In [12]:
%tensorboard --logs/func

# Masha-Allah it worked

UsageError: Line magic function `%tensorboard` not found.


* ##### **Creating a saved Model**
> The recommended way of sharing completely trained models is to use SavedModel. SavedModel contains both a collection of functions and a collection of weights.

In [13]:
# Saving the Model.. A separate directory is created to store the saved Model 
tf.saved_model.save(SeqModel, 'SavedModel')

# Reloading the saved Model. It is in Graph format with no knowledge of internal TensorFlow code.
SeqMod2 = tf.saved_model.load('SavedModel')

# Checking if the loaded model is an object of Sequential class 
isinstance(SeqMod2, dyn_SeqModel)


INFO:tensorflow:Assets written to: SavedModel\assets


False

> **The new model can only work with `pre-defined signatures` (input shape && data-type). It `cannot` be altered to perform on new signatures like Python Code.**

In [23]:
try:
    print(tx)
    # Sending in a known signature
    print(SeqMod2(tf.random.uniform([4,9])).numpy())
    
    # Sending an unknown argument
    print(SeqMod2(tf.constant(1.0)))
    
except:
    print("Error Raised due to incompatible input signature!!\n")


tf.Tensor(
[[24.571182 27.901814 16.977583 23.294868 26.34833  25.91249 ]
 [21.052631 27.666346 17.133547 25.036602 19.52958  17.714447]
 [17.855251 21.62644  28.017302 26.501816 26.499147 21.345549]
 [26.773119 27.076963 24.35641  18.492205 25.63773  24.267067]
 [20.255909 18.208706 16.620312 18.280846 19.88789  25.348614]
 [29.893362 26.71646  31.23438  22.951586 27.147898 28.046225]], shape=(6, 6), dtype=float32)
[[0.37259245 2.5588136  0.5820939 ]
 [2.8372762  2.243465   0.        ]
 [5.401466   3.54496    0.        ]
 [5.1961412  1.6135643  2.0946856 ]]
Some Error Raised!!
{e}}


### **Keras Models and Layers**

* ##### **Keras Layers**
> **Keras Layers as based on TF.Module. One can easily swap out TF.Module with Keras.layers.Layer**

In [28]:
class KerasDense(tf.keras.layers.Layer):
    # In_features defines the number of rows for the Layer_Weights Matrix 
    # Out_features determines the number of columns for the Weights Matrix
    def __init__(self, in_features, out_features, **args):
        super().__init__(**args)
        self.w = tf.Variable(
            tf.random.normal([in_features, out_features], name='Lyr_Weights')
        )
        # Here we are setting the model bias to zero (will try with identity matrix soon)..
        self.b = tf.Variable(
            tf.zeros([out_features], name='Lyr_Bias')
        )
    @tf.function
    def call(self, ten_x):
        return tf.nn.relu((ten_x @ self.w) + self.b)


denselayer = KerasDense(6, 5, name="Simple_Layer")
denselayer(tf.random.uniform([3, 6], minval=10, maxval=20))

<tf.Tensor: shape=(3, 5), dtype=float32, numpy=
array([[ 0.       , 10.594601 , 19.097565 ,  0.       ,  0.       ],
       [ 0.       ,  1.3583794, 23.31422  ,  5.3604546,  0.       ],
       [ 0.       ,  6.355938 , 24.87617  ,  1.0125513,  0.       ]],
      dtype=float32)>

* ##### **The build step**


* ##### **Keras Models**


### **Saving Keras Model**

### **`Sonnet`: Alternate deep-learning API for building ML Models**
> **By DeepMind,**
> **Based on tf.Module**